## Function that takes data and returns tips to user

In [1]:
# User inputs this data:
#   Resident/Visitor
#     Based on their selection, the proper presets are filled in.
#   Input location -> We will extract a blockID from this
#   For Visitors:
#     Dates of visit. This will give us the day of week as well.
# 
# Presets:
#   Resident:
#     1-year projection
#     Sun-Sat 12am - midnight
#
#   Visitor:
#     Dates of visit loaded from user input. This will give us day of week as well.
#     9am - 10pm

In [ ]:
# Output:
#  1. Crime rate in this area has decreased X% over last 5 years
#  2. Projected crime rate in this area is X% compared to city average
#  3. This area has higher/lower crime rate as the city average for      #   month    0 1 2 3 4 ... 11
#     X months of the year                                               # city_avg   1 1 1 1 1      1
#                                                                        # block_avg  2 2 2 2 2      2
#                                                                        # comp       h

BLOCK             = 0
BLOCK_5_YEARS_AGO = 1
BLOCK_PREDICTIONS = 2
CITY              = 3
CITY_PREDICTIONS  = 4

def perc_calc_block(blockid, data_old, data_new):
    v_old = data_old[0]
    v_new = data_new[0]
    
    if v_old == 0.0:
        print(f'Count for the {blockid} is zero')
        return np.Inf
    else:
        return (v_old - v_new) / v_old

# Summary for the resident:
#
#  1. Crime rate in this area has decreased X% over last 5 years
#     current crime rate - avg of latest year (2018)
#     crime rate 5 years ago                  (2013)
#
#     Output: If positive -> return crime 'increased'
#             If negative -> return crime 'decreased'

class GetData(object):
    def go(self, SESSION, data_type, blockid, year):
        #         if data_type == BLOCK_VISITOR:        # 
        #             SQL_QUERY = \
        #                 f'''
        #                     SELECT
        #                         COUNT(*)/AVG(blocks.population) AS crime_rate
        #                     FROM incident
        #                     INNER JOIN blocks ON incident.blockid = blocks.id
        #                     INNER JOIN crimetype ON incident.crimetypeid = crimetype.id
        #                         AND blocks.population > 0
        #                         AND incident.cityid = 1
        #                         AND incident.year = {year}
        #                         AND blocks.id = {blockid}
        #                         AND incident.dow >= {dow_start}
        #                         AND incident.dow <= {dow_end}
        #                         AND incident.hour >= {hour_start}
        #                         AND incident.hour <= {hour_end}
        #                 '''
            
            
        if data_type == BLOCK:
            SQL_QUERY = \
                f'''
                    SELECT
                        COUNT(*)/AVG(blocks.population) AS crime_rate
                    FROM incident
                    INNER JOIN blocks ON incident.blockid = blocks.id
                    INNER JOIN crimetype ON incident.crimetypeid = crimetype.id
                        AND blocks.population > 0
                        AND incident.cityid = 1
                        AND incident.year = {year}
                        AND blocks.id = {blockid}
                '''
        elif data_type == BLOCK_5_YEARS_AGO:
            SQL_QUERY = \
                f'''
                    SELECT
                        COUNT(*)/AVG(blocks.population) AS crime_rate
                    FROM incident
                    INNER JOIN blocks ON incident.blockid = blocks.id
                    INNER JOIN crimetype ON incident.crimetypeid = crimetype.id
                        AND blocks.population > 0
                        AND incident.cityid = 1
                        AND incident.year = {year-5}
                        AND blocks.id = {blockid}
                '''
        elif data_type == BLOCK_PREDICTIONS:
            SQL_QUERY = \
                f'''
                    SELECT
                        blocks.prediction AS predictions
                    FROM blocks
                        AND blocks.cityid = 1
                        AND blocks.id = {blockid}
                '''
        elif data_type == CITY:
            SQL_QUERY = \
                f'''
                    SELECT
                        COUNT(*)/(
                          SELECT SUM(blocks.population) AS city_population
                            FROM blocks) AS crime_rate
                    FROM incident
                    INNER JOIN block ON incident.blockid = blocks.id
                    INNER JOIN crimetype ON incident.crimetypeid = crimetype.id
                        AND blocks.population > 0
                        AND incident.cityid = 1
                        AND incident.year >= {year}
                        AND incident.year <= {year}
                '''
        elif data_type == CITY_PREDICTIONS:
            SQL_QUERY = \
                f'''
                    SELECT
                        blocks.prediction AS city_predictions
                    FROM blocks
                        AND blocks.cityid = 1
                '''
        else:
            raise ValueError('Unknown data type:', data_type)
    return SESSION.execute(text(SQL_QUERY)).fetchall()
    
    
    #  2. Projected crime rate in this area is X% compared to city average
    #     projected year crime rate (avg over the year for the block) vs.
    #     projected year crime rate (avg over the year for the city)
    #
    #     Output: For this block crime is X% higher/lower compared to city average
    
    #  3. This area has higher/lower crime rate as the city average for 
    #     (X months of the year)
    #
    #     Output: 
    #       if all months < city avg, say "This block is safer than the average city"
    #       elif any months > city avg, say "This block is unsafe than the average city during these months"
    #
    
    return result

In [ ]:
from contextlib import contextmanager

@contextmanager
def session_scope():
    """Provide a transactional scope around a series of operations."""

    DB_URI  = config('DB_URI')
    ENGINE  = create_engine(DB_URI)
    Session = sessionmaker(bind=ENGINE)
    SESSION = Session()
    
    try:
        yield SESSION
        SESSION.commit()
    except:
        SESSION.rollback()
        raise
    finally:
        SESSION.close()
        
    return 

In [ ]:
def tips_for_resident():
    with session_scope() as session:
        data_block_current = GetData().go(session, BLOCK, blockid, current_year)[0]
        data_block_5_years_ago = GetData().go(session, BLOCK_5_YEARS_AGO, blockid, current_year)[0]
        data_block_predictions = GetData().go(session, BLOCK_PREDICTIONS, blockid, None)[0]
        data_city_current = GetData().go(session, CITY, None, current_year)[0]
        data_city_predictions = GetData().go(session, CITY_PREDICTIONS, None, None)[0]
    
    #  1. Crime rate in this area has decreased X% over last 5 years
    perc_change_block = perc_calc_block(None, data_block_5_years_ago, data_block_current)
    block_str = "Crime rate in this area has " + "increased" + if perc_change_block > 0.0 else "decreased" + \
                str(np.absolute(perc_change_block)) + "% over last 5 years"
        
    #  2. Projected crime rate in this area is X% compared to city average
    perc_change_block_vs_city = perc_calc_block(None, data_block_predictions, data_city_predictions)
    block_vs_city_str = "Projected crime rate in this area is " + "higher by" + \
                            if perc_change_block > 0.0 else "lower by" + 
                            str(np.absolute(perc_change_block_vs_city)) + \
                            "% compared to city average"
    
    return [block_str, block_vs_city_str]

In [ ]:
# def tips_for_visitor():
#     with session_scope() as session:
#         data_block_current = GetData().go(session, BLOCK, blockid, current_year)[0]
#         data_block_5_years_ago = GetData().go(session, BLOCK_5_YEARS_AGO, blockid, current_year)[0]
#         data_block_predictions = GetData().go(session, BLOCK_PREDICTIONS, blockid, None)[0]
#         data_city_current = GetData().go(session, CITY, None, current_year)[0]
#         data_city_predictions = GetData().go(session, CITY_PREDICTIONS, None, None)[0]
    
#     #  1. Crime rate in this area has decreased X% over last 5 years
#     perc_change_block = perc_calc_block(None, data_block_5_years_ago, data_block_current)
#     block_str = "Crime rate in this area has " + "increased" + if perc_change_block > 0.0 else "decreased" + \
#                 str(np.absolute(perc_change_block)) + "% over last 5 years"
        
#     #  2. Projected crime rate in this area is X% compared to city average
#     perc_change_block_vs_city = perc_calc_block(None, data_block_predictions, data_city_predictions)
#     block_vs_city_str = "Projected crime rate in this area is " + "higher by" + \
#                             if perc_change_block > 0.0 else "lower by" + 
#                             str(np.absolute(perc_change_block_vs_city)) + \
#                             "% compared to city average"
    
#     return [block_str, block_vs_city_str]